# Pytorch : Classification Problem - Diabetics with NN

In [1]:
#import necessary libraries
#describe reason for import each libraries 
import numpy as np # converting data from pandas to torch
import torch 
import torch.nn as nn #main library to define the architecture of the neural network
import pandas as pd # to read the data from the csv file 
from sklearn.preprocessing import StandardScaler # used for feature normalization
from torch.utils.data import Dataset,DataLoader
import matplotlib.pyplot as plt # to plot loss with epochs

# Data Preprocessing

In [2]:
data = pd.read_csv('diabetes.csv')

In [3]:
data

,Number of times pregnant,Plasma glucose concentration,Diastolic blood pressure,Triceps skin fold thickness,2-Hour serum insulin,Body mass index,Age,Class
0,6,148,72,35,0,33.6,50,positive
1,1,85,66,29,0,26.6,31,negative
2,8,183,64,0,0,23.3,32,positive
3,1,89,66,23,94,28.1,21,negative
4,0,137,40,35,168,43.1,33,positive
...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,63,negative
764,2,122,70,27,0,36.8,27,negative
765,5,121,72,23,112,26.2,30,negative
766,1,126,60,0,0,30.1,47,positive


In [4]:
#Extract features X and o/p y from the data
X = data.iloc[:,:-1]
X = np.array(X)
y = data.iloc[:,-1]
y = np.array(y) # need to convert datatype into float else not possible to convert into tensor

In [5]:
y[y=='positive']=1.
y[y=='negative']=0.

In [6]:
y = np.array(y,dtype=np.float64)

In [7]:
y = y.reshape(len(y),1)

# Feature normalization

# Formula: $x^{\prime}=\frac{x-\mu}{\sigma}$ *where $\mu$ is mean and $\sigma$ is std

In [8]:
mean = X.mean(axis = 0) # taking mean along 
std = X.std(axis = 0)

In [9]:
X_norm = (X-mean)/std

In [10]:
X_norm

array([[ 0.63994726,  0.84832379,  0.14964075, ..., -0.69289057,
         0.20401277,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.69289057,
        -0.68442195, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -0.69289057,
        -1.10325546, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ...,  0.27959377,
        -0.73518964, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.69289057,
        -0.24020459,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.69289057,
        -0.20212881, -0.87137393]])

In [11]:
#alternate approach 
sc = StandardScaler()
X_norm1 = sc.fit_transform(X)

In [12]:
X_norm1

array([[ 0.63994726,  0.84832379,  0.14964075, ..., -0.69289057,
         0.20401277,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.69289057,
        -0.68442195, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -0.69289057,
        -1.10325546, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ...,  0.27959377,
        -0.73518964, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.69289057,
        -0.24020459,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.69289057,
        -0.20212881, -0.87137393]])

In [13]:
#Converting numpy array into tensor
X_tensor = torch.tensor(X_norm)
y_tensor = torch.tensor(y)

In [14]:
print(X_tensor.shape)
print(y_tensor.shape)

torch.Size([768, 7])
torch.Size([768, 1])


In [15]:
# We need to create custom dataset class to feed the data into dataloader
#because as per pytorch standard dataloader accepts dataset class
#this part can be copy pasted in case of use of custom data in X,y format
class Dataset(Dataset):
    def __init__(self,x,y):
        self.x = x
        self.y = y
        
    def __getitem__(self,index):
        # Get one item from the dataset
        return self.x[index], self.y[index]
    
    def __len__(self):
        return len(self.x)

In [16]:
dataset = Dataset(X_tensor,y_tensor)

In [17]:
print(dataset.__getitem__(2))
print(dataset.__len__())

(tensor([ 1.2339,  1.9437, -0.2639, -1.2882, -0.6929, -1.1033, -0.1056],
       dtype=torch.float64), tensor([1.], dtype=torch.float64))
768


In [18]:
#create the dataloader for the model
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [19]:
#Let's check the dataloader and iterate through it
print('Length of the dataloader:{}'.format(str(len(dataloader)))) # i.e no of batches
for (x,y) in dataloader:
    print("For one iteration (batch), there is:")
    print("Data:    {}".format(x.shape))
    print("Labels:  {}".format(y.shape))
    break  

Length of the dataloader:24
For one iteration (batch), there is:
Data:    torch.Size([32, 7])
Labels:  torch.Size([32, 1])


![demo](https://user-images.githubusercontent.com/30661597/60379583-246e5e80-9a68-11e9-8b7f-a4294234c201.png)

In [20]:
#create NN architecture as depicted above
class Model(nn.Module):
    def __init__(self,input_features,labels):
        super(Model, self).__init__() # if we dont include this then we will get error cannot assign module before Module.__init__() call
        self.input_features = input_features
        self.fc1 = nn.Linear(input_features,5)
        self.fc2 = nn.Linear(5,4)
        self.fc3 = nn.Linear(4,3)
        self.fc4 = nn.Linear(3,labels)
        self.sigmoid = nn.Sigmoid() # activation fn for the o/p
        self.tanh = nn.Tanh() # activation function for the hidden layers
        
    def forward(self,X):
        output = self.tanh(self.fc1(X))
        output = self.tanh(self.fc2(output))
        output = self.tanh(self.fc3(output))
        output = self.sigmoid(self.fc4(output))
        return output

**Reference only
# Note: we don't need to manually derive this, in pytorch we can use the cost function provided in the library

$H_{p}(q)=-\frac{1}{N} \sum_{i=1}^{N} y_{i} \cdot \log \left(p\left(y_{i}\right)\right)+\left(1-y_{i}\right) \cdot \log \left(1-p\left(y_{i}\right)\right)$


cost = -(Y * torch.log(hypothesis) + (1 - Y) * torch.log(1 - hypothesis)).mean()

In [78]:
def train(model,epochs,criterion,optimizer,dataloader):
    average_loss =[]
    
    for epoch in range(epochs):
        minibatch_loss=0
        for x,y in dataloader:
            x = x.float() # converting into float to avoid any error for type casting
            y = y.float()
            #forward propagation
            output = model(x)
            #calculate loss
            loss = criterion(output,y)
            minibatch_loss += loss
            #cleat gradient buffer # check the reason why the optimizer is zero
            optimizer.zero_grad()
            # Backward propagation
            loss.backward()
            # Weight Update: w <-- w - lr * gradient
            optimizer.step()
        #calculate accuracy for one epoch
        average_loss.append(minibatch_loss/len(dataloader)) 
        output = (output>0.5).float()
        #how many correct prection taking average of that
        accuracy = (output == y).float().mean()
        if (epoch+1)%50 ==0:
            print("Epoch {}/{}, Loss: {:.3f}, Accuracy: {:.3f}".format(epoch+1,epochs, average_loss[-1], accuracy))
    return [average_loss,model]

In [79]:
# call train function:
model = Model(X_tensor.shape[1],y_tensor.shape[1])
epochs = 150
# Loss function - Binary Cross Entropy 
#In Binary Cross Entropy: the input and output should have the same shape 
#size_average = True --> the losses are averaged over observations for each minibatch
#criterion = nn.BCELoss(size_average=True)
criterion = nn.BCELoss(reduction='mean')
#We will use SGD with momentum
# Need to read about torch modules that is getting used here
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
dataloader =dataloader
average_loss,model = train(model,epochs,criterion,optimizer,dataloader)

Epoch 50/150, Loss: 0.432, Accuracy: 0.750
Epoch 100/150, Loss: 0.418, Accuracy: 0.781
Epoch 150/150, Loss: 0.405, Accuracy: 0.625


In [27]:
def BCELossfn(output,y):
    loss = -(torch.sum(y*torch.log(output)+(1-y)*torch.log(1-output)))
    return loss
    

In [91]:
def predict(model,X,y):
    output = model(X)
    if  output>0.5:
        output = 1.
    else:
        output = 0.
    print('Predicted output:{}'.format(str(output)))
    print('Ground truth:{}'.format(str(y.item())))

In [92]:
predict(model,X_tensor[0].float(),y_tensor[0].float())

Predicted output:1.0
Ground truth:1.0
